# DomoAuth

> Fill in a module description here

In [141]:
# | default_exp DomoAuth


In [142]:
# | hide
from nbdev.showdoc import *
import fastcore.test as fctest


In [143]:
# | export
from fastcore.basics import patch_to
from dataclasses import dataclass, field
from abc import abstractmethod
import aiohttp
from typing import Optional, Union
import nbdev_domo.ResponseGetData as rgd


# Authentication Routes
Stand alone functions for users who prefer a functional programming approach

## Full Auth Route - username and password authentication

In [144]:
# | export
async def get_full_auth(
    domo_instance: str,  # domo_instance.domo.com
    domo_username: str,  # email address
    domo_password: str,
    session: Optional[aiohttp.ClientSession] = None,
) -> rgd.ResponseGetData:
    """uses username and password authentication to retrieve a full_auth access token"""

    is_close_session = False

    if not session:
        is_close_session = True
        session = aiohttp.ClientSession()

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    tokenHeaders = {"Content-Type": "application/json"}

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    res = await session.request(method="POST", url=url, headers=tokenHeaders, json=body)

    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_aiohttp_response(res)


#### Sample Implementations of get_full_auth

In [145]:
domo_instance = "domo-dojo"
domo_username = "test12@domo.com"
domo_password = "test1234"

res = await get_full_auth(domo_instance, domo_username, domo_password)
res


ResponseGetData(status=200, response={'success': False, 'reason': 'INVALID_CREDENTIALS'}, is_success=True, auth={})

In [146]:
# | hide
assert res.status == 200
assert res.response.get("success") == False


The 200 response confirms we were able to get a response from the credentials API, however success was FALSE because we sent invalid credentials

In [147]:
domo_instance = "test"
res = await get_full_auth(domo_instance, domo_username, domo_password)
res


ResponseGetData(status=403, response='Forbidden', is_success=False, auth={})

In [148]:
# | hide
assert res.status == 403


The 403 response is expected because test.domo.com presumeably does not exist or access if forbidden.

## Developer Auth Route - client_id and secret authentication

In [149]:
# | export
async def get_developer_auth(
    domo_client_id: str, domo_client_secret: str, session: Optional[aiohttp.ClientSession] = None
) -> rgd.ResponseGetData:

    """
    only use for authenticating against apis documented under developer.domo.com
    """
    is_close_session = False

    if not session:
        is_close_session = True
        session = aiohttp.ClientSession(
            auth=aiohttp.BasicAuth(domo_client_id, domo_client_secret)
        )

    url = f"https://api.domo.com/oauth/token?grant_type=client_credentials"

    res = await session.request(method="GET", url=url)

    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_aiohttp_response(res)


#### Sample Implementations of get_developer_auth
The 401 response is expected because we are using invalid credentials

In [150]:
domo_client_id = 'test_id'
domo_client_secret = 'test_secret'

await get_developer_auth(domo_client_id, domo_client_secret)


ResponseGetData(status=401, response='Unauthorized', is_success=False, auth={})

## Access Token Auth Route - access_token authentication
This access_token based authentication assumes the user has been provided a valid access token from Domo > Admin > Authentication > Access Token so authentication routes are actually not required.

Per the Domo JavaCLI implementation, users can test the validity of the access_token agains the 'me' API

!! Note about the Me API !!
It appears that access_token authentication will direct the 

In [151]:
# | export
async def test_access_token(domo_access_token: str,  # as provided in Domo > Admin > Authentication > AccessTokens
                            domo_instance: str,  # <domo_instance>.domo.com
                            session: Optional[aiohttp.ClientSession] = None
                            ):
    """
    will attempt to validate against the 'me' API.  
    This is the same authentication test the Domo Java CLI uses.
    """

    is_close_session = False

    if not session:
        is_close_session = True
        session = aiohttp.ClientSession()

    url = f"https://{domo_instance}.domo.com/api/content/v2/users/me"

    tokenHeaders = {"X-DOMO-Developer-Token": domo_access_token}

    res = await session.request(method="GET", headers=tokenHeaders, url=url)

    if is_close_session:
        await session.close()

    return await rgd.ResponseGetData._from_aiohttp_response(res)


#### Sample implementation of `test_access_token`

In [152]:
domo_instance = 'playstation-beta'
domo_access_token = 'a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c'

await test_access_token(domo_instance=domo_instance, domo_access_token=domo_access_token)


ResponseGetData(status=401, response='Unauthorized', is_success=False, auth={})

# Classes

Different Domo Auth classes will have a variety of required vs optional parameters.  To avoid multiple initialization and post_intialization statements, we mix multiple classes together such that classes with optional parameters are mixed in before classes with required parameters.

In [153]:
# | exporti
@dataclass
class _DomoAuth_Required:
    """required parameters for all Domo Auth classes"""

    domo_instance: str

    def __post_init__(self):
        if domo_instance:
            self.set_manual_login()

    def set_manual_login(self):
        self.url_manual_login = f"https://{self.domo_instance}.domo.com/auth/index?domoManualLogin=true"


@dataclass
class _DomoAuth_Optional:
    """parameters are defined after initialization"""

    token: Optional[str] = field(default=None, repr=False)
    token_name: Optional[str] = field(default=None)
    user_id: Optional[str] = field(default=None, repr=False)
    auth_header: dict = field(default_factory=dict, repr=False)

    url_manual_login: Optional[str] = None

    async def get_auth_token(self) -> Union[str, None]:
        """placeholder method"""
        pass

    async def generate_auth_header(self) -> Union[dict, None]:
        """returns auth header appropriate for this authentication method"""
        pass


In [154]:
# | export
@dataclass
class DomoAuth(_DomoAuth_Optional, _DomoAuth_Required):
    """abstract DomoAuth class"""

    pass


In [155]:
# Attributes of `DomoAuth`
domo_instance = 'test'
[attr for attr in dir(DomoAuth(domo_instance)) if not attr.startswith("__")]


['auth_header',
 'domo_instance',
 'generate_auth_header',
 'get_auth_token',
 'set_manual_login',
 'token',
 'token_name',
 'url_manual_login',
 'user_id']

In [156]:
# validate can print manual login link
domo_instance = "test"

da = DomoAuth(domo_instance)
da.url_manual_login


'https://test.domo.com/auth/index?domoManualLogin=true'

## DomoAuth Error Classes

In [157]:
# | exporti
class DomoErrror(Exception):
    """base exception"""

    def __init__(self, status: Optional[int] = None,  # API request status
                 message: str = "error",  # <domo_instance>.domo.com
                 domo_instance: Optional[str] = None
                 ):

        instance_str = f" at {domo_instance}" if domo_instance else ""
        status_str = f"Status {status} - " if status else ""
        self.message = f"{status_str}{message}{instance_str}"
        super().__init__(self.message)


In [158]:
# | export
class InvalidCredentialsError(DomoErrror):
    """return invalid credentials sent to API"""

    def __init__(
        self,
        status: Optional[int] = None,  # API request status
        message="invalid credentials",
        domo_instance: Optional[str] = None
    ):

        super().__init__(status=status, message=message, domo_instance=domo_instance)


class InvalidInstanceError(DomoErrror):
    """return if invalid domo_instance sent to API"""

    def __init__(
        self, status: Optional[int] = None, message="invalid instance", domo_instance: Optional[str] = None
    ):
        super().__init__(status=status, message=message, domo_instance=domo_instance)


In [159]:
# | hide
# validates that Errors can compile
fctest.test_eq(isinstance(InvalidCredentialsError(),
               InvalidCredentialsError), True)
fctest.test_eq(isinstance(InvalidCredentialsError(),
               InvalidCredentialsError), True)
fctest.test_eq(isinstance(InvalidInstanceError(), InvalidInstanceError), True)


## DomoFullAuth

In [160]:
# | exporti
@dataclass
class _DomoFullAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_username: str
    domo_password: str = field(repr=False)

In [161]:
# | export
@dataclass
class DomoFullAuth(_DomoAuth_Optional, _DomoFullAuth_Required):
    """use for full authentication token"""

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"x-domo-authentication": token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> str:
        """returns `token` if valid credentials provided else raises Exception and returns None"""

        res = await get_full_auth(
            domo_instance=self.domo_instance,
            domo_username=self.domo_username,
            domo_password=self.domo_password,
            session=session,
        )

        if res.is_success and res.response.get('reason') == 'INVALID_CREDENTIALS':
            raise InvalidCredentialsError(
                status=res.status,
                message=str(res.response.get("reason")),
                domo_instance=self.domo_instance,
            )

        if res.status == 403:
            raise InvalidInstanceError(
                status=res.status,
                message="INVALID INSTANCE",
                domo_instance=self.domo_instance
            )

        token = str(res.response.get("sessionToken"))
        self.token = token
        self.user_id = str(res.response.get("userId"))

        self.auth_header = self.generate_auth_header(token=token)

        if not self.token_name:
            self.token_name = 'full_auth'

        return self.token


#### sample implementations of DomoFullAuth

In [162]:
domo_instance = "domo-dojo"
domo_username = "test12@domo.com"
domo_password = "test1234"

try:
    full_auth = DomoFullAuth(domo_instance, domo_username, domo_password)
    res = await full_auth.get_auth_token()

except InvalidCredentialsError as e:
    print(e)


Status 200 - INVALID_CREDENTIALS at domo-dojo


In [163]:
domo_instance = "test"

try:
    full_auth = DomoFullAuth(domo_instance, domo_username, domo_password)
    await full_auth.get_auth_token()
except InvalidInstanceError as e:
    print(e)


Status 403 - INVALID INSTANCE at test


In [164]:
# | hide
fctest.test_eq(isinstance(
    DomoAuth(domo_instance=domo_instance), DomoAuth), True)

fctest.test_eq(
    isinstance(
        DomoFullAuth(
            domo_instance=domo_instance,
            domo_password=domo_password,
            domo_username=domo_username,
        ),
        DomoFullAuth,
    ),
    True,
)


## DomoTokenAuth

In [165]:
# | exporti
@dataclass
class _DomoTokenAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_access_token: str = field(repr=False)


In [166]:
# | export
@dataclass
class DomoTokenAuth(_DomoAuth_Optional, _DomoTokenAuth_Required):
    """
    use for access_token authentication.
    Tokens are generated in domo > admin > access token
    Necessary in cases where direct sign on is not permitted
    """

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"x-domo-developer-token": token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[aiohttp.ClientSession] = None
    ) -> str:
        """
        updates internal attributes
        having an access_token assumes pre-authenticaiton
        """

        res = await test_access_token(
            domo_instance=self.domo_instance,
            domo_access_token=self.domo_access_token,
            session=session,
        )

        if res.status == 401 and res.response == 'Unauthorized':
            raise InvalidCredentialsError(
                status=res.status,
                message=res.response,
                domo_instance=self.domo_instance,
            )

        self.token = self.domo_access_token
        self.user_id = res.response.get("id")

        self.auth_header = self.generate_auth_header(token=self.token)

        if not self.token_name:
            self.token_name = 'token_auth'

        return self.token

#### Sample implementation of DomoTokenAuth

In [167]:
domo_instance = 'test'
domo_access_token = 'test_access_token'

domo_access_token = 'a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c'
try:
    domo_auth = DomoTokenAuth(domo_instance, domo_access_token)
    token = await domo_auth.get_auth_token()
    print(domo_auth)

except InvalidCredentialsError as e:
    print(e)


DomoTokenAuth(domo_instance='test', token_name='token_auth', url_manual_login='https://test.domo.com/auth/index?domoManualLogin=true')


## DomoDeveloperAuth

In [168]:
# | exporti
@dataclass
class _DomoDeveloperAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_client_id: str
    domo_client_secret: str = field(repr=False)


In [169]:
# | export
@dataclass(init=False)
class DomoDeveloperAuth(_DomoAuth_Optional, _DomoDeveloperAuth_Required):
    """use for full authentication token"""

    def __init__(self, domo_client_id: str, domo_client_secret: str):
        self.domo_client_id = domo_client_id
        self.domo_client_secret = domo_client_secret
        self.domo_instance = ''

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"Authorization": "bearer " + token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> str:

        res = await get_developer_auth(
            domo_client_id=self.domo_client_id,
            domo_client_secret=self.domo_client_secret,
            session=session,
        )

        if res.status == 401:
            raise InvalidCredentialsError(
                status=res.status,
                message=str(res.response),
                domo_instance=self.domo_instance,
            )

        token = str(res.response.get("access_token"))
        self.token = token
        self.user_id = res.response.get("userId")
        self.domo_instance = res.response.get('domain')
        self.set_manual_login()

        self.auth_header = self.generate_auth_header(token=token)

        if not self.token_name:
            self.token_name = 'developer_auth'

        return token


#### Sample implementations of DomoDeveloperAuth

In [170]:
domo_client_id = 'test_client'
domo_client_secret = 'test_secret'

try:
    domo_auth = DomoDeveloperAuth(domo_client_id, domo_client_secret)
    await domo_auth.get_auth_token()
except InvalidCredentialsError as e:
    print(e)


Status 401 - Unauthorized


In [171]:
# | hide
import nbdev

nbdev.nbdev_export()
